In [1]:
pip install cassandra-driver

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cassandra; print (cassandra.__version__)

3.25.0


In [2]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cloud_config= {
        'secure_connect_bundle': 'C:/Users/amr01/Downloads/secure-connect-ass22.zip'
}
auth_provider = PlainTextAuthProvider('vouDifKrZeEqtKJiuPDIOFRU', 'jQz8KSd0aZN9mqH1ywQ6KTZSgg6lFv1nqRZwn3CtDOOt6HmXZF92ihzXBAGZky8bEtxlw_.v8-E_70EufJefMLlSehwN.7q7f02zh5gOiZxgyIvT-MJYKiZlji21d_7H')
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
    print(row[0])
else:
    print("An error occurred.")

4.0.0.6816


In [3]:
import pandas as pd
df = pd.read_csv('D:/computer science GIU/semester 6/BigData/GIU_2718_56_7524_2022-05-17T10_13_53/datasets/taxi_trip_data.csv',nrows = 2000,parse_dates=["pickup_datetime", "dropoff_datetime"])
df_geo = pd.read_csv('D:/computer science GIU/semester 6/BigData/GIU_2718_56_7524_2022-05-17T10_13_53/datasets/taxi_zone_geo.csv',nrows = 2000)
i,j = df.shape

In [4]:
session.set_keyspace('ass2')
row = cluster.metadata.keyspaces['ass2']
from pprint import pprint
pprint(vars(row))

{'aggregates': {},
 'durable_writes': True,
 'functions': {},
 'graph_engine': None,
 'indexes': {},
 'name': 'ass2',
 'replication_strategy': <cassandra.metadata.NetworkTopologyStrategy object at 0x000001732B3D7220>,
 'tables': {'ass2': <cassandra.metadata.TableMetadataV3 object at 0x000001732B3D76D0>},
 'user_types': {},
 'views': {}}


In [5]:
session.execute("""
    create table if not exists ass2.ass2 (
        vendor_id text,
        pickup_datetime datetime,
        dropoff_datetime datetime,
        passenger_count int,
        trip_distance float,
         
        payment_type text,
        fare_amount float,
        extra float,
        mta_tax float,
        tip_amount float,
        tolls_amount float,
        imp_surcharge float,
        pickup_location_id text,
        dropoff_location_id text,
        trip_id int,
        Primary key (vendor_id)
     );
""")

In [1]:
for x in range(0,1000):
    session.execute("insert into ass2.ass2(vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,imp_surcharge,pickup_location_id,dropoff_location_id) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);",
    (
        str(df.loc[x].vendor_id),
        str(df.loc[x].pickup_datetime),
        str(df.loc[x].dropoff_datetime),
        int(df.loc[x].passenger_count),
        float(df.loc[x].trip_distance),
        str(df.loc[x].payment_type),
        float(df.loc[x].fare_amount),
        float(df.loc[x].extra),
        float(df.loc[x].mta_tax), 
        float(df.loc[x].tip_amount),
        float(df.loc[x].tolls_amount), 
        float(df.loc[x].imp_surcharge),
        str(df.loc[x].pickup_location_id), 
        str(df.loc[x].dropoff_location_id) ))

In [2]:
show_rows = session.execute("select * from ass2.ass2 limit 1")
for rows in show_rows:
    print(rows)

In [3]:
session.execute("ALTER TABLE ass2.ass2 add duration float;")

In [13]:
session.execute("ALTER TABLE ass2.ass2  DROP id;")

In [4]:
row = session.execute(" select * from ass2.ass2 ")
for c in row:
    duration = (c[1]-c[10]).seconds/60
    session.execute("update ass2.ass2 set duration = {0} where vendor_id='{1}';".format(duration,c[0]))
    print(duration)

In [5]:
session.execute("ALTER TABLE ass2.ass2 add total_cost float;")

In [6]:
row = session.execute("select * from ass2.ass2 ")
for c in row:
    total_cost = (c[5] + c[4] + c[7] + c[12] + c[13] + c[6])
    session.execute("update ass2.ass2 set total_cost = {0} where vendor_id='{1}';".format(total_cost,c[0]))
    print(total_cost)

In [7]:
avg_tip=[]
for i in range(2000): 
    if(df['passenger_count'][i]==1):
        avg_tip1 = df['tip_amount'][i].mean()
    if(df['passenger_count'][i]==2):
        avg_tip2 = df['tip_amount'][i].mean()
    if(df['passenger_count'][i]==3):
        avg_tip3 = df['tip_amount'][i].mean()
    if(df['passenger_count'][i]==4):
        avg_tip4 = df['tip_amount'][i].mean()
    if(df['passenger_count'][i]==5):
        avg_tip5 = df['tip_amount'][i].mean()
    if(df['passenger_count'][i]==6):
        avg_tip6 = df['tip_amount'][i].mean()
avg_tip=[avg_tip1,avg_tip2,avg_tip3,avg_tip4,avg_tip5,avg_tip6]
avg_tip

In [11]:
morning = []
afternoon = []
evening = []
for i in range (0, 2000):
    if(df.loc[i].pickup_datetime.hour >= 5 and df.loc[i].pickup_datetime.hour < 12):
        morning.append(df.iloc[i]['payment_type'])
    if(df.loc[i].pickup_datetime.hour >= 12 and df.loc[i].pickup_datetime.hour < 17):
        afternoon.append(df.iloc[i]['payment_type'])
    if (df.loc[i].pickup_datetime.hour>= 17 and df.loc[i].pickup_datetime.hour < 21):
            evening.append(df.iloc[i]['payment_type'])

In [9]:
import numpy as np
a = np.array(morning)
unique, counts = np.unique(a, return_counts=True)
dict(zip(unique, counts))

In [10]:
a = np.array(afternoon)
unique, counts = np.unique(a, return_counts=True)
dict(zip(unique, counts))

In [11]:
a = np.array(evening)
unique, counts = np.unique(a, return_counts=True)
dict(zip(unique, counts))

In [12]:
df['pickup_location_id'].nlargest(n=5)

In [13]:
import findspark
from pyspark import SparkContext
from pyspark import SparkConf
spark.sql("select zone_id,zone_name,COUNT(*) as num_trip from cassandratest2.cassandratest GROUP BY zone_id,zone_name ").show(5)
spark.sql("SELECT time_day,payment_type,COUNT(*) as num_trips FROM cassandratest2.cassandratest GROUP BY time_day,payment_type ")